# HEAPML Project
## Random Forest

In [1]:
### GENERAL IMPORTS ###
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### PYMATGEN/MATMINER IMPORTS ###
from matminer.featurizers import composition as cf
from matminer.featurizers.base import MultipleFeaturizer

### SKLEARN IMPORTS ###
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score, train_test_split, RepeatedStratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, precision_score, recall_score

### SKOMPTOMIZE IMPORTS ###
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.plots import plot_convergence

### 1. Import Featurized Data

In [2]:
feature_calculators = MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset('magpie')])
feature_labels = feature_calculators.feature_labels()

alloys = pd.read_csv('./data/featurized_alloys.csv')

display(alloys)

,formula,phase,composition_obj,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,MagpieData minimum Number,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,AgAlCoCrCuNi,3,Ag1 Al1 Co1 Cr1 Cu1 Ni1,6,0.408248,0.302853,0.238495,0.215285,0.199372,13.0,...,1.548471,0.357311,0.476415,0.0,194.0,229.0,35.0,220.500000,8.833333,194.0
1,AgCoCrFeMnNi,1,Ag1 Co1 Cr1 Fe1 Mn1 Ni1,6,0.408248,0.302853,0.238495,0.215285,0.199372,24.0,...,2.110663,0.709140,0.746951,0.0,194.0,229.0,35.0,219.833333,9.555556,194.0
2,Al0.02CoCrFeMnNi,1,Al0.02 Co1 Cr1 Fe1 Mn1 Ni1,6,0.445450,0.340633,0.274847,0.250697,0.233988,13.0,...,2.110663,0.847577,0.782462,0.0,194.0,229.0,35.0,218.824701,10.617292,194.0
3,Al0.03CoCrFeMnNi,1,Al0.03 Co1 Cr1 Fe1 Mn1 Ni1,6,0.444586,0.339956,0.274300,0.250199,0.233523,13.0,...,2.110663,0.845892,0.782246,0.0,194.0,229.0,35.0,218.836978,10.605947,194.0
4,Al0.04CoCrFeMnNi,1,Al0.04 Co1 Cr1 Fe1 Mn1 Ni1,6,0.443735,0.339282,0.273756,0.249702,0.233059,13.0,...,2.110663,0.844214,0.782026,0.0,194.0,229.0,35.0,218.849206,10.594608,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,Zr2.0TiHfVNb2.0,1,Zr2 Ti1 Hf1 V1 Nb2,5,0.473804,0.381200,0.331220,0.315980,0.306266,22.0,...,0.000023,0.000003,0.000006,0.0,194.0,229.0,35.0,209.000000,17.142857,194.0
1069,ZrTiHfCuNiFe,1,Zr1 Ti1 Hf1 Cu1 Ni1 Fe1,6,0.408248,0.302853,0.238495,0.215285,0.199372,22.0,...,2.110663,0.451013,0.601344,0.0,194.0,229.0,35.0,210.166667,16.166667,194.0
1070,ZrTiHfNb0.5Mo0.5,1,Zr1 Ti1 Hf1 Nb0.5 Mo0.5,5,0.467707,0.370312,0.312720,0.292700,0.279049,22.0,...,0.000023,0.000006,0.000008,0.0,194.0,229.0,35.0,202.750000,13.125000,194.0
1071,ZrTiHfNb0.5Ta0.5,1,Zr1 Ti1 Hf1 Nb0.5 Ta0.5,5,0.467707,0.370312,0.312720,0.292700,0.279049,22.0,...,0.000023,0.000006,0.000008,0.0,194.0,229.0,35.0,202.750000,13.125000,194.0


### 2. Generate Dataset
*Formula, phase and composition_obj columns are removed from $X$*.

In [3]:
x_cols = [c for c in alloys.columns if c not in ['formula', 'phase', 'composition_obj']]

y = alloys['phase'].values
X = alloys[x_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, stratify=y)

display(X)

,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,6,0.408248,0.302853,0.238495,0.215285,0.199372,13.0,47.0,34.0,28.000000,...,1.548471,0.357311,0.476415,0.0,194.0,229.0,35.0,220.500000,8.833333,194.0
1,6,0.408248,0.302853,0.238495,0.215285,0.199372,24.0,47.0,23.0,29.500000,...,2.110663,0.709140,0.746951,0.0,194.0,229.0,35.0,219.833333,9.555556,194.0
2,6,0.445450,0.340633,0.274847,0.250697,0.233988,13.0,28.0,15.0,25.948207,...,2.110663,0.847577,0.782462,0.0,194.0,229.0,35.0,218.824701,10.617292,194.0
3,6,0.444586,0.339956,0.274300,0.250199,0.233523,13.0,28.0,15.0,25.922465,...,2.110663,0.845892,0.782246,0.0,194.0,229.0,35.0,218.836978,10.605947,194.0
4,6,0.443735,0.339282,0.273756,0.249702,0.233059,13.0,28.0,15.0,25.896825,...,2.110663,0.844214,0.782026,0.0,194.0,229.0,35.0,218.849206,10.594608,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,5,0.473804,0.381200,0.331220,0.315980,0.306266,22.0,72.0,50.0,39.857143,...,0.000023,0.000003,0.000006,0.0,194.0,229.0,35.0,209.000000,17.142857,194.0
1069,6,0.408248,0.302853,0.238495,0.215285,0.199372,22.0,72.0,50.0,36.166667,...,2.110663,0.451013,0.601344,0.0,194.0,229.0,35.0,210.166667,16.166667,194.0
1070,5,0.467707,0.370312,0.312720,0.292700,0.279049,22.0,72.0,50.0,43.875000,...,0.000023,0.000006,0.000008,0.0,194.0,229.0,35.0,202.750000,13.125000,194.0
1071,5,0.467707,0.370312,0.312720,0.292700,0.279049,22.0,73.0,51.0,47.750000,...,0.000023,0.000006,0.000008,0.0,194.0,229.0,35.0,202.750000,13.125000,194.0


### 3. Train Model

In [4]:
rf = RandomForestClassifier(random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=0)

### 4. Evaluate Model

In [5]:
y_pred = rf.predict(X_test)

print('Accuracy: %.6f' % accuracy_score(y_test, y_pred))
print('Precision: %.6f' % precision_score(y_test, y_pred, average='macro'))
print('Recall: %.6f' % recall_score(y_test, y_pred, average='macro'))

Accuracy: 0.729577
Precision: 0.686640
Recall: 0.685465


### 5. Feature Selection

In [6]:
permutation_importance = permutation_importance(rf, X_train, y_train, n_repeats=20, random_state=0, scoring='accuracy', n_jobs=-1)

p_i = sorted(zip(feature_labels, permutation_importance.importances_mean), key=lambda x: x[1], reverse=True)
p_i = pd.DataFrame(p_i, columns=['Label', 'Mean Score'])

display(p_i)

,Label,Mean Score
0,2-norm,0.001184
1,MagpieData avg_dev Column,0.000975
2,MagpieData avg_dev NsUnfilled,0.000696
3,MagpieData mode SpaceGroupNumber,0.000696
4,MagpieData avg_dev NUnfilled,0.000557
...,...,...
133,MagpieData minimum Electronegativity,-0.000348
134,MagpieData mean GSvolume_pa,-0.000348
135,MagpieData maximum NdUnfilled,-0.000348
136,MagpieData mean NdValence,-0.000487


### 6. Regenerate Dataset

In [7]:
feature_count = 27

feature_selection = p_i['Label'].head(feature_count).values

x_cols = [c for c in alloys.columns if c in feature_selection]

y = alloys['phase'].values
X = alloys[x_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, stratify=y)

display(X)
for feature in feature_selection:
    print(feature)

,2-norm,3-norm,MagpieData maximum Number,MagpieData mode MeltingT,MagpieData mean Column,MagpieData avg_dev Column,MagpieData minimum CovalentRadius,MagpieData range CovalentRadius,MagpieData mode CovalentRadius,MagpieData maximum Electronegativity,...,MagpieData mean NsUnfilled,MagpieData avg_dev NsUnfilled,MagpieData mean NpUnfilled,MagpieData avg_dev NdUnfilled,MagpieData mean NUnfilled,MagpieData avg_dev NUnfilled,MagpieData minimum GSvolume_pa,MagpieData maximum GSvolume_pa,MagpieData avg_dev GSmagmom,MagpieData mode SpaceGroupNumber
0,0.408248,0.302853,47.0,933.47,10.000000,1.666667,121.0,24.0,121.0,1.93,...,0.500000,0.500000,0.833333,1.666667,3.000000,1.666667,10.245,16.480,0.476415,194.0
1,0.408248,0.302853,47.0,1234.93,8.500000,1.500000,124.0,21.0,124.0,1.93,...,0.333333,0.444444,0.000000,1.500000,3.500000,1.500000,10.245,16.330,0.746951,194.0
2,0.445450,0.340633,28.0,1519.00,8.019920,1.219028,121.0,18.0,124.0,1.91,...,0.199203,0.319043,0.019920,1.053952,4.003984,1.199981,10.245,16.480,0.782462,194.0
3,0.444586,0.339956,28.0,1519.00,8.029821,1.228415,121.0,18.0,124.0,1.91,...,0.198807,0.318566,0.029821,1.060832,4.005964,1.199957,10.245,16.480,0.782246,194.0
4,0.443735,0.339282,28.0,1519.00,8.039683,1.237717,121.0,18.0,124.0,1.91,...,0.198413,0.318090,0.039683,1.067649,4.007937,1.199924,10.245,16.480,0.782026,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,0.473804,0.381200,72.0,2128.00,4.428571,0.489796,153.0,22.0,164.0,1.63,...,0.285714,0.408163,0.000000,0.816327,7.571429,0.489796,13.010,23.195,0.000006,194.0
1069,0.408248,0.302853,72.0,1357.77,6.833333,2.833333,124.0,51.0,124.0,1.91,...,0.166667,0.277778,0.000000,3.000000,5.166667,2.833333,10.320,23.195,0.601344,194.0
1070,0.467707,0.370312,72.0,1941.00,4.375000,0.562500,154.0,21.0,160.0,2.16,...,0.250000,0.375000,0.000000,0.937500,7.625000,0.562500,15.690,23.195,0.000008,194.0
1071,0.467707,0.370312,73.0,1941.00,4.250000,0.375000,160.0,15.0,160.0,1.60,...,0.125000,0.218750,0.000000,0.562500,7.750000,0.375000,16.690,23.195,0.000008,194.0


2-norm
MagpieData avg_dev Column
MagpieData avg_dev NsUnfilled
MagpieData mode SpaceGroupNumber
MagpieData avg_dev NUnfilled
MagpieData mean NUnfilled
MagpieData minimum CovalentRadius
MagpieData mode MeltingT
MagpieData maximum Number
MagpieData mean NpUnfilled
MagpieData maximum NValence
MagpieData avg_dev NValence
MagpieData minimum GSvolume_pa
MagpieData mean Column
MagpieData maximum Electronegativity
MagpieData avg_dev NsValence
3-norm
MagpieData mean NsValence
MagpieData maximum NdValence
MagpieData avg_dev GSmagmom
MagpieData range CovalentRadius
MagpieData mean NsUnfilled
MagpieData avg_dev Electronegativity
MagpieData range NdValence
MagpieData maximum GSvolume_pa
MagpieData avg_dev NdUnfilled
MagpieData mode CovalentRadius


### 7. Retrain Model

In [8]:
rf = RandomForestClassifier(random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=0)

### 8. Re-evaluate Model

In [9]:
y_pred = rf.predict(X_test)

print('Accuracy: %.6f' % accuracy_score(y_test, y_pred))
print('Precision: %.6f' % precision_score(y_test, y_pred, average='macro'))
print('Recall: %.6f' % recall_score(y_test, y_pred, average='macro'))

Accuracy: 0.729577
Precision: 0.691797
Recall: 0.676982


### 9. Tune Hyperparameters

In [ ]:
space = [Integer(10, 10**5, 'log-uniform', name='n_estimators'), 
         Integer(1, 10**5, 'log-uniform', name='max_depth'), 
         Integer(2, 10**3, 'uniform', name='min_samples_split'), 
         Integer(1, 10**3, 'uniform', name='min_samples_leaf'), 
         Real(10**-7, 0.5, 'log-uniform', name='min_weight_fraction_leaf'), 
         Integer(1, 138, 'uniform', name='max_features'), 
         Integer(2, 10**5, 'log-uniform', name='max_leaf_nodes'), 
         Real(10**-5, 10**-1, 'log-uniform', name='ccp_alpha'), 
         Integer(1, 138, 'uniform', name='max_samples')]

In [ ]:
@use_named_args(space)
def objective(**params):
    rf = RandomForestClassifier(random_state=0, n_jobs=-1)
    rf.set_params(**params)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
    accuracy = cross_val_score(rf, X_train, y_train, cv=cv, scoring='accuracy')
    accuracy = np.mean(accuracy)
    return 1.0 - accuracy

class tqdm_skopt(object):
    def __init__(self, **kwargs):
        self._bar = tqdm(**kwargs)
        
    def __call__(self, res):
        self._bar.update()

In [ ]:
n_calls = 100
result = gp_minimize(objective,
                     space,
                     n_calls=n_calls,
                     random_state=0,
                     callback=[tqdm_skopt(total=n_calls, desc="Progress")])

plot_convergence(result)
plt.savefig('plot.png')

print(result.x)
print(1-result.fun)

### 10. Retrain Model

In [ ]:
rf = RandomForestClassifier(n_estimators=1000,
                            max_depth=1000,
                            min_samples_split=2,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=1e-05,
                            max_features=17,
                            max_leaf_nodes=681,
                            ccp_alpha=0.0017758739729781885,
                            max_samples=138,
                            random_state=0)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print('Accuracy: %.6f' % accuracy_score(y_test, y_pred))
print('Precision: %.6f' % precision_score(y_test, y_pred, average='macro'))
print('Recall: %.6f' % recall_score(y_test, y_pred, average='macro'))